In [1]:
import os
import re
import io
import sys
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, roc_auc_score, average_precision_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

from dotenv import load_dotenv, find_dotenv

In [ ]:
# FUNÇÕES DE PREPARO

def coerce_numeric(s):
    return pd.to_numeric(s, errors="coerce")
def extrair_fase(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).lower()
    if "alfa" in valor:
        return 0
    m = re.search(r"fase\s*(\d+)", valor)
    if m:
        return int(m.group(1))
    return np.nan


def padronizar_genero(df):
    df = df.copy()
    if "genero" in df.columns:
        df["genero"] = df["genero"].astype(str).str.strip().str.lower()
        map_genero = {
            "menino": "masculino",
            "masculino": "masculino",
            "menina": "feminino",
            "feminino": "feminino"
        }
        df["genero"] = df["genero"].map(map_genero)
    return df


def padronizar_idade(df):
    df = df.copy()
    if "idade" not in df.columns:
        return df

    s = df["idade"]
    dt = pd.to_datetime(s, errors="coerce")

    idade_from_date = np.where(
        dt.notna() & (dt.dt.year == 1900) & (dt.dt.month == 1),
        dt.dt.day,
        np.nan
    )

    idade_num = pd.to_numeric(s, errors="coerce")
    idade_final = pd.Series(idade_num, index=df.index)

    mask = idade_final.isna() & ~pd.isna(idade_from_date)
    idade_final.loc[mask] = idade_from_date[mask]

    # faixa plausível
    idade_final = idade_final.where(idade_final.between(6, 30))
    df["idade"] = idade_final.round()
    return df


def tratar_inde_2024(df):
    df = df.copy()
    if "inde_2024" in df.columns:
        tmp = df["inde_2024"].astype(str).str.strip().str.upper()
        tmp = tmp.replace("INCLUIR", np.nan)
        df["inde_2024"] = coerce_numeric(tmp)
    return df


def preparar_base(df, modo_treino: bool):
    """
    modo_treino=True:
      - cria target a partir de 'ian'
      - remove vazamento (ian/defasagem)
    modo_treino=False:
      - não cria target
      - remove vazamento se existir
    """
    df = df.copy()

    df = padronizar_genero(df)
    df = padronizar_idade(df)
    df = tratar_inde_2024(df)

    if "fase_ideal" in df.columns:
        df["fase_ideal"] = df["fase_ideal"].apply(extrair_fase)

    # Target (somente treino)
    if modo_treino:
        if "ian" not in df.columns:
            raise ValueError("modo_treino=True exige coluna 'ian' para criar a target.")
        df["risco_defasagem_atual"] = (pd.to_numeric(df["ian"], errors="coerce") <= 5).astype(int)

    # Remover vazamento sempre que existir
    df = df.drop(columns=[c for c in ["ian", "defasagem"] if c in df.columns], errors="ignore")

    # Remoções iguais ao treino (mantém seu padrão)
    colunas_para_remover = ["ra",
                            "nome",
                            "data_nasc",
                            "escola",
                            "avaliador_1",
                            "avaliador_2",
                            "avaliador_3",
                            "avaliador_4",
                            "avaliador_5",
                            "avaliador_6",
                            "rec_av1",
                            "rec_av2",
                            "rec_av3",
                            "rec_av4",
                            "rec_av5",
                            "rec_av6",
                            "rec_psicologia",
                            "indicado",
                            "atingiu_pv",
                            "destaque_ieg",
                            "destaque_ida",
                            "destaque_ivp",
                            "pedra_2020",
                            "pedra_2021",
                            "pedra_2022",
                            "pedra_2023",
                            "pedra_2024",
                            "fase",
                            "turma",
                            "instituicao_ensino",
                            "ativo_inativo",
                            "cg",
                            "cf",
                            "ct",
                            "inde_2024"
    ]
    df = df.drop(columns=[c for c in colunas_para_remover if c in df.columns], errors="ignore")

    # Features extras
    cols_acad = [c for c in ["mat","por","ing"] if c in df.columns]
    if len(cols_acad) >= 2:
        df["media_academica"] = df[cols_acad].mean(axis=1)

    cols_comp = [c for c in ["iaa","ieg","ips","ipp"] if c in df.columns]
    if len(cols_comp) >= 2:
        df["media_comportamental"] = df[cols_comp].mean(axis=1)

    if ("inde_2022" in df.columns) and ("inde_2023" in df.columns):
        df["delta_inde"] = df["inde_2023"] - df["inde_2022"]

    return df


def make_preprocess(X_train: pd.DataFrame) -> ColumnTransformer:
    num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in X_train.columns if c not in num_cols]

    numeric_pipe = Pipeline(steps=[
        ("imputer", KNNImputer(n_neighbors=7, weights="distance")),
        ("scaler", StandardScaler())
    ])

    categorical_pipe = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", categorical_pipe, cat_cols),
        ],
        remainder="drop"
    )
    return preprocess

In [3]:
# 2) TREINO COMPLETO + TESTE + THRESHOLD + SALVAR ARQUIVOS
# ============================================================

def treinar_e_salvar(excel_path: str,
                    out_model_path: str = "modelo_passos_magicos.pkl",
                    out_cfg_path: str = "config_passos_magicos.pkl",
                    seed: int = 42):

    base = pd.read_excel(excel_path)
    print("Shape original:", base.shape)

    base2 = preparar_base(base, modo_treino=True)
    print("Shape após preparo:", base2.shape)
    print("Target balance:\n", base2["risco_defasagem_atual"].value_counts())

    if "ano_pede" not in base2.columns:
        raise ValueError("Coluna 'ano_pede' não encontrada após preparo. Precisa dela para split temporal.")

    # Split temporal
    train_df = base2[base2["ano_pede"] < 2024].copy()
    test_df  = base2[base2["ano_pede"] == 2024].copy()

    X_train = train_df.drop(columns=["risco_defasagem_atual"])
    y_train = train_df["risco_defasagem_atual"].astype(int)

    X_test = test_df.drop(columns=["risco_defasagem_atual"])
    y_test = test_df["risco_defasagem_atual"].astype(int)

    print("Treino:", X_train.shape, "Teste:", X_test.shape)

    preprocess = make_preprocess(X_train)

    models = {
        "LogReg": LogisticRegression(max_iter=2000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(
            n_estimators=400, random_state=seed,
            class_weight="balanced_subsample"
        ),
        "MLP": MLPClassifier(
            hidden_layer_sizes=(64, 32),
            activation="relu",
            solver="adam",
            max_iter=500,
            random_state=seed
        ),
    }

    if HAS_XGB:
        models["XGBoost"] = XGBClassifier(
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=seed,
            eval_metric="logloss"
        )
    else:
        print("XGBoost não disponível. Seguindo sem XGBoost.")

    # CV no treino
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scoring = {"acc": "accuracy", "roc_auc": "roc_auc", "pr_auc": "average_precision"}

    print("\n================= CV (TREINO) =================")
    cv_results = []
    pipes = {}

    for name, clf in models.items():
        pipe = Pipeline(steps=[("prep", preprocess), ("model", clf)])
        pipes[name] = pipe

        scores = cross_validate(
            pipe, X_train, y_train,
            cv=cv, scoring=scoring,
            n_jobs=-1, return_train_score=False
        )

        row = {
            "model": name,
            "acc_mean": scores["test_acc"].mean(),
            "acc_std": scores["test_acc"].std(),
            "roc_auc_mean": scores["test_roc_auc"].mean(),
            "roc_auc_std": scores["test_roc_auc"].std(),
            "pr_auc_mean": scores["test_pr_auc"].mean(),
            "pr_auc_std": scores["test_pr_auc"].std(),
        }
        cv_results.append(row)

        print(f"\n{name}")
        print(f"  ACC     : {row['acc_mean']:.4f} ± {row['acc_std']:.4f}")
        print(f"  ROC AUC : {row['roc_auc_mean']:.4f} ± {row['roc_auc_std']:.4f}")
        print(f"  PR AUC  : {row['pr_auc_mean']:.4f} ± {row['pr_auc_std']:.4f}")

    cv_df = pd.DataFrame(cv_results).sort_values("roc_auc_mean", ascending=False)
    print("\nResumo CV:")
    print(cv_df)

    # Teste final 2024
    print("\n================= TESTE FINAL (2024) =================")
    test_rows = []

    def avaliar(nome, pipe):
        pipe.fit(X_train, y_train)
        y_proba = pipe.predict_proba(X_test)[:, 1]
        y_pred = pipe.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba)
        pr  = average_precision_score(y_test, y_proba)

        print(f"\n>>> {nome}")
        print(f"ACC     : {acc:.4f}")
        print(f"ROC AUC : {roc:.4f}")
        print(f"PR AUC  : {pr:.4f}")
        print("\nMatriz de confusão:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification report:")
        print(classification_report(y_test, y_pred, digits=4))

        return {"model": nome, "acc": acc, "roc_auc": roc, "pr_auc": pr}

    for name, pipe in pipes.items():
        test_rows.append(avaliar(name, pipe))

    test_df_res = pd.DataFrame(test_rows).sort_values("roc_auc", ascending=False)
    print("\nResumo TESTE (ordenado por ROC AUC):")
    print(test_df_res)

    best_name = test_df_res.iloc[0]["model"]
    best_pipe = pipes[best_name]
    print("\nMelhor modelo no TESTE:", best_name)

    # Ajuste threshold no MELHOR modelo do teste
    best_pipe.fit(X_train, y_train)
    y_proba_best = best_pipe.predict_proba(X_test)[:, 1]

    thresholds = np.linspace(0.10, 0.90, 17)
    thr_rows = []
    for t in thresholds:
        y_pred_t = (y_proba_best >= t).astype(int)
        report = classification_report(y_test, y_pred_t, output_dict=True, zero_division=0)
        acc = accuracy_score(y_test, y_pred_t)
        prec1 = report["1"]["precision"]
        rec1 = report["1"]["recall"]
        f11 = report["1"]["f1-score"]
        thr_rows.append([t, acc, prec1, rec1, f11])

    thr_df = pd.DataFrame(thr_rows, columns=["threshold","accuracy","precision_risco","recall_risco","f1_risco"])
    print("\nTabela de thresholds (classe RISCO=1):")
    print(thr_df)

    best_thr_row = thr_df.sort_values("f1_risco", ascending=False).iloc[0]
    best_threshold = float(best_thr_row["threshold"])

    print("\nMelhor threshold (por F1 da classe 1):", best_threshold)
    print("Linha escolhida:\n", best_thr_row)

    # Salvar arquivos
    joblib.dump(best_pipe, out_model_path)
    joblib.dump({"threshold": best_threshold, "best_model": best_name}, out_cfg_path)

    print("\nArquivos salvos:")
    print(f"- (pipeline completo)")
    print(f"- (threshold + nome do modelo)")

    return best_name, best_threshold, cv_df, test_df_res, thr_df

In [ ]:
# Localiza e carrega as variáveis do arquivo .env automaticamente
load_dotenv(find_dotenv())

# Puxa o caminho da variável de ambiente
data_path = os.getenv("DATA_PATH")
models = os.getenv("MODELS")

MODO = "treino"  # "treino" ou "app"
EXCEL_PATH = data_path 
MODEL_PATH = f"{models}modelo_passos_magicos.pkl"
CFG_PATH = f"{models}config_passos_magicos.pkl"

if MODO == "treino":
    treinar_e_salvar(EXCEL_PATH, MODEL_PATH, CFG_PATH)

elif MODO == "app":
    # Streamlit NÃO é ideal no Colab, mas em ambiente local funciona:
    # streamlit run seu_arquivo.py
    print("Modo app: rode em ambiente local com Streamlit.")
    print("Exemplo: streamlit run projeto_completo.py")
else:
    raise ValueError("MODO inválido. Use 'treino' ou 'app'.")

python-dotenv could not parse statement starting at line 15


Shape original: (3030, 18)
Shape após preparo: (3030, 19)
Target balance:
 risco_defasagem_atual
1    1687
0    1343
Name: count, dtype: int64
Treino: (1874, 18) Teste: (1156, 18)

================= CV (TREINO) =================

LogReg
  ACC     : 0.8927 ± 0.0096
  ROC AUC : 0.9394 ± 0.0101
  PR AUC  : 0.9568 ± 0.0118

RandomForest
  ACC     : 0.8389 ± 0.0189
  ROC AUC : 0.9122 ± 0.0093
  PR AUC  : 0.9340 ± 0.0092

MLP
  ACC     : 0.9899 ± 0.0062
  ROC AUC : 0.9969 ± 0.0022
  PR AUC  : 0.9986 ± 0.0009

XGBoost
  ACC     : 0.9370 ± 0.0132
  ROC AUC : 0.9772 ± 0.0081
  PR AUC  : 0.9827 ± 0.0077

Resumo CV:
          model  acc_mean   acc_std  roc_auc_mean  roc_auc_std  pr_auc_mean  \
2           MLP  0.989861  0.006174      0.996912     0.002174     0.998643   
3       XGBoost  0.937045  0.013200      0.977159     0.008055     0.982677   
0        LogReg  0.892740  0.009631      0.939360     0.010099     0.956788   
1  RandomForest  0.838863  0.018940      0.912180     0.009282     0.93

In [5]:
pipe = joblib.load(f'{models}modelo_passos_magicos.pkl')
cfg = joblib.load(f'{models}config_passos_magicos.pkl')

threshold = cfg['threshold']
print('Threshold: ', threshold)

base = pd.read_excel(data_path)

base_prep = preparar_base(base, modo_treino=False)

sample = base_prep.sample(10, random_state=42)

proba = pipe.predict_proba(sample)[:, 1]
pred = (proba >= threshold).astype(int)

sample['proba'] = proba
sample['pred'] = pred

processed_path = os.getenv("PROCESSED")
output_file = os.path.join(processed_path, "sample_predicoes.xlsx")
sample.to_excel(output_file, index=False)

sample

Threshold:  0.1


,Unnamed: 0,ano_pede,inde_2022,inde_2023,n_av,iaa,ieg,ips,ipp,ida,...,por,ing,ipv,fase_ideal,risco_defasagem_atual,media_academica,media_comportamental,delta_inde,proba,pred
1207,1207,2023,NaN,5.973450,2.0,10.000,9.300000,2.520,5.3125,3.90,...,3.3,NaN,6.500000,NaN,1,3.900000,6.783125,NaN,0.975335,1
256,256,2024,NaN,6.478200,2.0,10.002,5.389610,5.635,7.1875,2.00,...,4.0,NaN,4.250000,NaN,1,2.000000,7.053528,NaN,0.000027,0
2356,2356,2022,7.928000,NaN,4.0,8.800,8.900000,7.500,NaN,5.80,...,7.2,4.8,7.917000,NaN,0,5.766667,8.400000,NaN,0.263769,1
175,175,2024,NaN,NaN,2.0,9.502,10.000000,7.510,7.5000,8.25,...,6.5,NaN,7.500000,NaN,1,8.250000,8.628000,NaN,0.729123,1
211,211,2024,NaN,6.861700,3.0,8.502,9.328063,4.380,5.3125,7.25,...,8.5,NaN,5.783333,NaN,0,7.250000,6.880641,NaN,0.010062,0
411,411,2024,7.038889,5.823133,3.0,9.002,7.595238,4.385,5.6250,3.00,...,3.0,NaN,5.996667,NaN,0,3.000000,6.651810,-1.215756,0.986001,1
52,52,2024,NaN,7.079700,3.0,9.002,9.473684,7.510,8.1250,7.25,...,7.5,NaN,6.943333,NaN,1,7.250000,8.527671,NaN,0.978572,1
266,266,2024,7.736100,7.680200,3.0,10.002,8.484848,6.260,8.1250,8.50,...,7.0,NaN,8.056667,NaN,0,8.500000,8.217962,-0.055900,0.878799,1
479,479,2024,NaN,NaN,4.0,8.502,8.782609,7.510,6.8750,4.75,...,5.5,NaN,6.827500,NaN,1,4.750000,7.917402,NaN,0.000110,0
1292,1292,2023,NaN,8.017200,2.0,9.500,8.100000,7.520,6.2500,8.90,...,8.6,NaN,6.500000,NaN,0,8.850000,7.842500,NaN,0.169649,1


In [6]:
list(pipe.named_steps.keys())

['prep', 'model']

In [7]:
model = pipe.named_steps['model']

if hasattr(model, 'feature_importances_'):
    importances = model.feature_importances_
elif hasattr(model, 'coef_'):
    importances = np.abs(model.coef_[0])
else:
    importances = None
    print("Este modelo não suporta extração de importância de variáveis.")

if importances is not None:
    features = pipe.named_steps['prep'].get_feature_names_out()
    imp = pd.DataFrame({
        'feature': features,
        'importance': importances
    }).sort_values('importance', ascending=False)

    print(imp.head(15))

                      feature  importance
2              num__inde_2022    5.271833
3              num__inde_2023    4.102627
5                    num__iaa    3.401025
1               num__ano_pede    2.667250
6                    num__ieg    2.246977
7                    num__ips    2.179712
0             num__Unnamed: 0    1.853061
16            num__delta_inde    1.797100
15  num__media_comportamental    1.598446
13                   num__ipv    1.416159
8                    num__ipp    1.412128
9                    num__ida    0.725197
10                   num__mat    0.645902
14       num__media_academica    0.548667
11                   num__por    0.497636


In [ ]:
# ==========================================
# 1. Carregar Modelo, Configuração e Dados
# ==========================================

pipe = joblib.load(f'{models}modelo_passos_magicos.pkl')
cfg = joblib.load(f'{models}config_passos_magicos.pkl')

threshold = cfg['threshold']
print('Threshold: ', threshold)

base = pd.read_excel(data_path)

# ==========================================
# 2. Gerar a Amostra Bruta e Fazer Predições
# ==========================================
sample_raw = base.sample(10, random_state=42).copy()

# A predição acontece nos bastidores
base_prep = preparar_base(sample_raw, modo_treino=False)
proba = pipe.predict_proba(base_prep)[:, 1]
pred = (proba >= threshold).astype(int)

# ==========================================
# 3. Organizar as Colunas EXATAMENTE como no App
# ==========================================
# Esta é a sequência visual do seu App (Lendo da esquerda para a direita, linha por linha)
ordem_visual_app = [
    # --- 1. Dados do Aluno ---
    'idade', 
    'genero', 
    'fase_ideal', 
    
    # --- 2. Notas Acadêmicas ---
    'mat', 
    'por', 
    'ing', 
    
    # --- 3. Indicadores (Lendo lado a lado no Streamlit) ---
    'iaa',         # Coluna 1
    'ips',         # Coluna 2
    'ieg',         # Coluna 1
    'ipp',         # Coluna 2
    'inde_2024',   # Coluna 1 (Se não existir, vamos tentar mapear do 'inde' ou 'ian')
    
    # --- Histórico ---
    'inde_2022', 
    'inde_2023'
]

sample_final = pd.DataFrame()

# Loop inteligente para procurar as colunas (ignora maiúsculas/minúsculas)
for col_app in ordem_visual_app:
    # Procura se a coluna existe na base bruta com alguma variação de nome
    col_encontrada = next((c for c in sample_raw.columns if c.lower() == col_app.lower()), None)
    
    if col_encontrada:
        sample_final[col_app] = sample_raw[col_encontrada]
    else:
        # Tenta um "plano B" para o INDE 2024 caso na base ele se chame apenas 'inde' ou 'ian'
        if col_app == 'inde_2024' and 'inde' in sample_raw.columns:
            sample_final[col_app] = sample_raw['inde']
        else:
            sample_final[col_app] = np.nan # Se realmente não existir, fica vazio

# Adiciona os resultados da predição no final
sample_final['proba_modelo'] = proba
sample_final['predicao_risco'] = pred

# Exibe na tela do Notebook
display(sample_final)

# ==========================================
# 4. Exportar para Excel
# ==========================================
processed_path = os.getenv("PROCESSED", ".") 
output_file = os.path.join(processed_path, "sample_predicoes_app.xlsx")
sample_final.to_excel(output_file, index=False)

print(f"\nArquivo exportado com sucesso")

Threshold:  0.1


,idade,genero,fase_ideal,mat,por,ing,iaa,ips,ieg,ipp,inde_2024,inde_2022,inde_2023,proba_modelo,predicao_risco
1207,NaN,NaN,3,4.5,3.3,NaN,10.000,2.520,9.300000,5.3125,NaN,NaN,5.973450,0.975335,1
256,NaN,NaN,3,0.0,4.0,NaN,10.002,5.635,5.389610,7.1875,NaN,NaN,6.478200,0.000027,0
2356,NaN,NaN,3,5.3,7.2,4.8,8.800,7.500,8.900000,NaN,NaN,7.928000,NaN,0.263769,1
175,NaN,NaN,1,10.0,6.5,NaN,9.502,7.510,10.000000,7.5000,NaN,NaN,NaN,0.729123,1
211,NaN,NaN,1,6.0,8.5,NaN,8.502,4.380,9.328063,5.3125,NaN,NaN,6.861700,0.010062,0
411,NaN,NaN,2,3.0,3.0,NaN,9.002,4.385,7.595238,5.6250,NaN,7.038889,5.823133,0.986001,1
52,NaN,NaN,2,7.0,7.5,NaN,9.002,7.510,9.473684,8.1250,NaN,NaN,7.079700,0.978572,1
266,NaN,NaN,1,10.0,7.0,NaN,10.002,6.260,8.484848,8.1250,NaN,7.736100,7.680200,0.878799,1
479,NaN,NaN,3,4.0,5.5,NaN,8.502,7.510,8.782609,6.8750,NaN,NaN,NaN,0.000110,0
1292,NaN,NaN,0,9.1,8.6,NaN,9.500,7.520,8.100000,6.2500,NaN,NaN,8.017200,0.169649,1



Arquivo exportado com sucesso para: C:\Users\ricar\OneDrive\Cursos\Pós Graduação\Data Analytics - FIAP\07 - Fase 5 - Deep Learning and Unstructured Data\projeto_datathon\data\processed\sample_predicoes_app.xlsx


In [11]:
data_path = os.getenv("RAWS_DATA_PATH")

# ==========================================
# 1. Carregar Modelo e Configurações
# ==========================================
pipe = joblib.load(f'{models}modelo_passos_magicos.pkl')
cfg = joblib.load(f'{models}config_passos_magicos.pkl')

threshold = cfg['threshold']
print('Threshold do Modelo: ', threshold)

# ==========================================
# 2. Carregar a Base Bruta de 2024
# ==========================================
# Se você tiver lendo do CSV que enviou, use read_csv. 
# Se for do Excel original, use pd.read_excel('BASE DE DADOS PEDE...xlsx', sheet_name='PEDE2024')
df_2024 = pd.read_excel(data_path)

# Pega uma amostra de 10 alunos aleatórios
sample_raw = df_2024.sample(10, random_state=42).copy()

# ==========================================
# 3. Criar a Amostra EXATAMENTE como no App (Mesma ordem)
# ==========================================
# Vamos mapear as colunas originais (com letras maiúsculas/espaços) para as 
# variáveis minúsculas que o seu app.py usa.
sample_app = pd.DataFrame()

# 1. Dados do Aluno
sample_app['idade'] = sample_raw['Idade']
sample_app['genero'] = sample_raw['Gênero']
sample_app['fase_ideal'] = sample_raw['Fase Ideal']

# 2. Notas Acadêmicas
sample_app['mat'] = sample_raw['Mat']
sample_app['por'] = sample_raw['Por']
sample_app['ing'] = sample_raw['Ing']

# 3. Indicadores (Ordem visual do app: IAA, IPS, IEG, IPP, INDE 2024)
sample_app['iaa'] = sample_raw['IAA']
sample_app['ips'] = sample_raw['IPS']
sample_app['ieg'] = sample_raw['IEG']
sample_app['ipp'] = sample_raw['IPP']
sample_app['inde_2024'] = sample_raw['INDE 2024']

# Histórico
sample_app['inde_2022'] = sample_raw['INDE 22']
sample_app['inde_2023'] = sample_raw['INDE 23']

# ==========================================
# 4. Preparar para Predição (Igual ao que o App faz)
# ==========================================
# O modelo precisa daquelas colunas extras (mesmo que vazias) para o Imputer funcionar,
# exatamente como você programou no seu app.py!
sample_para_modelo = sample_app.copy()
sample_para_modelo['ano_pede'] = 2024
sample_para_modelo['ipv'] = np.nan
sample_para_modelo['ida'] = np.nan
sample_para_modelo['ano_ingresso'] = np.nan
sample_para_modelo['n_av'] = np.nan

# Passamos pela função preparar_base (mesmo processo do Streamlit)
base_prep = preparar_base(sample_para_modelo, modo_treino=False)

# Extraímos a probabilidade e a classe (0 ou 1)
proba = pipe.predict_proba(base_prep)[:, 1]
pred = (proba >= threshold).astype(int)

# ==========================================
# 5. Adicionar Resultados e Exportar
# ==========================================
sample_app['proba_modelo'] = proba
sample_app['predicao_risco'] = pred

# Exibe na tela do Jupyter
display(sample_app)

# Exporta para Excel
processed_path = os.getenv("PROCESSED", ".")
output_file = os.path.join(processed_path, "sample_predicoes_app_real.xlsx")
sample_app.to_excel(output_file, index=False)

print(f"\n✅ Arquivo de validação criado com sucesso em: {output_file}")

Threshold do Modelo:  0.1


ValueError: Invalid file path or buffer object type: <class 'NoneType'>